In [1]:
import numpy as np
import pandas as pd
from matplotlib import rcParams
import matplotlib.pyplot as plt
from IPython.display import clear_output
from scipy.stats import norm


rcParams["savefig.dpi"] = 200
rcParams["figure.dpi"] = 400
rcParams["font.size"] = 18

In [2]:
loc_lcparam = 'datos/Pantheon+SH0ES.dat.txt'
loc_lcparam_sys = 'datos/Pantheon+SH0ES_STAT+SYS.cov.txt'

In [3]:
zup = 0.05  # upper limit of the redshift range
zdown = 0.01  # lower limit of the redshift range0

In [5]:
import numpy as np
import pandas as pd

# Load the headers from the first row of the file
with open(loc_lcparam) as f:
    header = f.readline().strip().split()

# Read data from files into pandas DataFrames
lcparam_df = pd.read_csv(loc_lcparam, delimiter=' ', usecols=[header[i] for i in [2, 8, 9, 10, 11, 26, 27, 12, 13]])
lcparam_sys_df = pd.read_csv(loc_lcparam_sys, delimiter=' ')

# Filter all light curves within the redshift range 0.1 > z > 0.001
filtered_df = lcparam_df[(lcparam_df['zHD'] < zup) & (lcparam_df['zHD'] > zdown)]
ind = filtered_df.index

# Extract specific columns and assign to variables
zz = filtered_df['zHD'].values
mz = filtered_df['m_b_corr'].values
sigmz = filtered_df['m_b_corr_err_DIAG'].values
muz = filtered_df['MU_SH0ES'].values
sigmuz = filtered_df['MU_SH0ES_ERR_DIAG'].values
ra = filtered_df['RA'].values
dec = filtered_df['DEC'].values
muceph = filtered_df['CEPH_DIST'].values
hostyn = filtered_df['IS_CALIBRATOR'].values
hostyn_ind = np.where(hostyn == 1)[0]  # Get indices where hostyn equals 1

# Reshape and select subset of elements in lcparam_sys_df
cov_z = lcparam_sys_df.values.reshape(1701, 1701)[np.ix_(ind, ind)]
inv_cov_z = np.linalg.inv(cov_z)

# Print information
# print("We have", len(zz), f"SNe with {zdown} < z < {zup}")
# print("\nCov matrix shape:", cov_z.shape)
print(f"{len(hostyn)} Light Curves with {zdown} < z < {zup}:")

# Create DataFrame
cov_mat = pd.DataFrame(cov_z, columns=range(len(zz)), index=range(len(zz)))

534 Light Curves with 0.01 < z < 0.05:


In [6]:
has_cepheid = lcparam_df[lcparam_df['CEPH_DIST'] != -9]

filtradas = has_cepheid[has_cepheid['zHD'] > 0.01]

print(f"""We have {len(filtradas)} Cepheid host distances with  0.1 > zHD > 0.01""")

We have 10 Cepheid host distances with  0.1 > zHD > 0.01


In [7]:
def dl_data(mu: float) -> float:
    return 10**((mu - 25.0) / 5.0)


def calculate_dlz_error(sigmuz: np.ndarray, mean_mu: float) -> float:
    dl_deriv = (1/5) * dl_data(mean_mu) * np.log(10)
    return np.abs(dl_deriv) * sigmuz

In [12]:
dlz = dl_data(muz)
sigdlz = calculate_dlz_error(sigmuz, np.mean(muz))

dlz

array([ 35.96499465,  38.33893106,  39.21750856,  46.51146297,
        44.16314568,  45.26473686,  42.14247133,  41.43620757,
        47.43730413,  45.56380662,  41.84272884,  41.08279666,
        43.26931302,  51.78214065,  55.81359543,  53.38017583,
        58.4709298 ,  48.60937084,  49.99654574,  52.07868384,
        50.05874991,  59.65680632,  55.012325  ,  55.98349402,
        58.30690645,  55.15691913,  55.58018654,  52.17710752,
        50.53124516,  56.21341891,  53.37034375,  50.18339052,
        55.85988021,  56.51191183,  51.88478244,  51.69636417,
        50.5824662 ,  49.46978239,  62.17846551,  62.64984488,
        60.94526924,  57.46454866,  56.71004606,  56.80675741,
        56.62132134,  54.30252197,  54.87316369,  57.68461521,
        54.07792262,  58.09249108,  58.73270456,  63.64144033,
        63.82340937,  73.67497548,  68.48886987,  70.85981269,
        60.47278918,  68.3187639 ,  56.30150446,  59.77505715,
        60.96211137,  56.37155306,  55.43448281,  60.02

In [11]:
num_points = len(dlz)
num_subsets = 4
subset_size = num_points // num_subsets

dlz_subsets = [dlz[i * subset_size:(i + 1) * subset_size] for i in range(num_subsets)]
sigdlz_subsets = [sigdlz[i * subset_size:(i + 1) * subset_size] for i in range(num_subsets)]


def plot_dlz_subsets(dlz_subset, sigdlz_subset, subset_xy_lim, plot_number):
    x_values = np.arange(len(dlz_subset))
    plt.errorbar(x_values, dlz_subset, yerr=sigdlz_subset, fmt='-o', color='green', capsize=3)
    plt.ylabel('dlz')
    plt.xlabel('Elements')
    # plt.xlim(0, {subset_xy_lim[0]+ 30})
    # plt.ylim(0, {subset_xy_lim[1]+ 30})
    plt.title(f'$D_L$ Subset for elements between {plot_number*len(dlz_subset)} and {(plot_number+1)*len(dlz_subset)}')
    plt.grid(True)


plt.figure(figsize=(20, 30))

plt.subplot(4, 1, 1)
plot_dlz_subsets(dlz_subsets[0], sigdlz_subsets[0], np.max(dlz_subsets[0]) + 20, 0)

plt.subplot(4, 1, 2)
plot_dlz_subsets(dlz_subsets[1], sigdlz_subsets[1], np.max(dlz_subsets[1]) + 20, 1)

plt.subplot(4, 1, 3)
plot_dlz_subsets(dlz_subsets[2], sigdlz_subsets[2], np.max(dlz_subsets[2]) + 20, 2)

plt.subplot(4, 1, 4)
plot_dlz_subsets(dlz_subsets[3], sigdlz_subsets[3], np.max(dlz_subsets[3]) + 20, 3)